# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `warehouse_and_retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per supplier.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [69]:
import numpy as np
import pandas as pd
from scipy import stats

data=pd.read_csv('../Warehouse_and_Retail_Sales.csv')
data.head(20)
data.shape



(128355, 9)

In [70]:
#【Clean】Check null values in the dataframe, 24 suppliers are null, 1 item type is null.It makes no sense that they are null, drop them


null_sup=data.loc[data['SUPPLIER'].isnull()].index
null_item=data.loc[data['ITEM TYPE'].isnull()].index
    
data=data.drop(null_sup)
data=data.drop(null_item)
    
data.isnull().sum()



YEAR                0
MONTH               0
SUPPLIER            0
ITEM CODE           0
ITEM DESCRIPTION    0
ITEM TYPE           0
RETAIL SALES        0
RETAIL TRANSFERS    0
WAREHOUSE SALES     0
dtype: int64

In [71]:
# 【Clean】It is impossible that the sales value are negative，turn negative to positive

data['WAREHOUSE SALES'] = data['WAREHOUSE SALES'].abs() 
data['RETAIL SALES']=data['RETAIL SALES'].abs() 
data['RETAIL TRANSFERS'] =data['RETAIL TRANSFERS'].abs()  


# Capitalize the first letter in the column
data['ITEM DESCRIPTION'] = data['ITEM DESCRIPTION'].str.capitalize()

data['SUPPLIER'] = data['SUPPLIER'].str.title()

# 【Clean】I tried to drop the outliers, but 

# Q1 = data.quantile(0.25)
# Q3 = data.quantile(0.75)
# IQR = Q3 - Q1

# data = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]
# data.shape
# data.head()

# A table for cleaned data
data.to_csv('../clean_table.csv',index=False)

In [72]:
# A table for the aggregate per supplier.
set(data['SUPPLIER'])
data_sup=data[['SUPPLIER','YEAR','MONTH','ITEM TYPE','ITEM CODE','ITEM DESCRIPTION','RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES']]
data_sup.describe()



#based on suppliers, aggregate 3 different columns and compute their sum
table_sup=data_sup.groupby(['SUPPLIER'])['RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES'].sum().reset_index()



# Sum up the total sales values, and create a new column name 'SUM'
table_sup['SUM']=table_sup.sum(axis=1)



# Change other 3 columns names
table_sup=table_sup.rename(columns={'RETAIL SALES':'TOTAL RETAIL SALES','RETAIL TRANSFERS':'TOTAL RETAIL TRANSFERS','WAREHOUSE SALES':'TOTAL WAREHOUSE SALES'})
table_sup.columns


# Rank the 'SUPPLIER' based on SUM of all sales values
table_sup=table_sup.sort_values(by='SUM', ascending=False)

table_sup.to_csv('../supplier_table.csv',index=False)






/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


In [73]:
# A table for the aggregate per item.

# Check the types of items
set(data['ITEM TYPE'])


# Change the order of 'ITEM TYPE' to the first column
data_item=data[['ITEM TYPE','ITEM DESCRIPTION','ITEM CODE','SUPPLIER','YEAR','MONTH','RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES']]
data_item.head()


# Based on ITEM TYPE, aggregate 3 different columns and compute their sum
table_item=data_item.groupby(['ITEM TYPE'])['RETAIL SALES','RETAIL TRANSFERS','WAREHOUSE SALES'].sum().reset_index()


# To sum up the total sales value of each ITEM TYPE
table_item['SUM']=table_item.sum(axis=1)


# To change the names of 3 columns
table_item=table_item.rename(columns={'RETAIL SALES':'TOTAL RETAIL SALES','RETAIL TRANSFERS':'TOTAL RETAIL TRANSFERS','WAREHOUSE SALES':'TOTAL WAREHOUSE SALES'})


# To ranke the most popular ITEM TYPE
table_item.sort_values(by='SUM', ascending=False)


# To create a table 
table_item.to_csv('../item_table.csv',index=False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]
